In [1]:
%load_ext autoreload
%autoreload 2
import csv
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from hpe_library.lib_import import *
from hpe_library. my_utils import *
os.chdir('/home/hrai/codes/MotionBERT/')

In [2]:
data = readpkl('./action_recognition_result.pkl')

In [3]:
data.keys()

dict_keys(['gt', 'output', 'split'])

In [6]:
data['gt'].shape, data['output'].shape, data['split'].shape

((7405,), (7405, 7), (7405,))

In [7]:
target = torch.from_numpy(data['gt'])
output = torch.from_numpy(data['output'])

In [12]:
def accuracy(output, target, topk=(1,)):
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)
        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res
accuracy(output, target)

[tensor([95.7731])]

In [51]:
subject_list = np.array([x.split('_')[0] for x in data['split']])
action_list = np.array([x.split('_')[1] for x in data['split']])
dict_action_list = {}
for action in set(action_list):
    dict_action_list[action] = np.where(action_list == action)[0]
    #print(len(dict_action_list[action]))
    
for sub in range(11, 19):
    sub_list = np.where(subject_list == f'S{sub}')[0]
    
    # subject-wise accuracy
    target = torch.from_numpy(data['gt'][sub_list])
    output = torch.from_numpy(data['output'][sub_list])
    acc = accuracy(output, target)
    print(f'subject {sub}, num subject: {len(sub_list)}, total acc: {acc[0].item()}')
    # action-wise accuracy
    for key in dict_action_list.keys():
        action_list = np.intersect1d(sub_list, dict_action_list[key])
        target = torch.from_numpy(data['gt'][action_list])
        output = torch.from_numpy(data['output'][action_list])
        acc = accuracy(output, target)
        print(key, len(action_list), acc[0].item())
    print('------------------------------------------')
    

subject 11, num subject: 965, total acc: 97.09844970703125
pushup 125 77.5999984741211
squat 100 100.0
burpee 130 100.0
kneeup 115 100.0
yoga 175 100.00000762939453
cat 175 100.00000762939453
benchpress 145 100.0
------------------------------------------
subject 12, num subject: 965, total acc: 97.30570220947266
pushup 95 95.78947448730469
squat 110 97.2727279663086
burpee 140 96.42857360839844
kneeup 90 100.00000762939453
yoga 150 100.0
cat 205 93.65853881835938
benchpress 175 99.42857360839844
------------------------------------------
subject 13, num subject: 990, total acc: 86.16161346435547
pushup 135 0.0
squat 110 100.0
burpee 125 100.0
kneeup 115 100.0
yoga 155 100.0
cat 150 99.33333587646484
benchpress 200 99.5
------------------------------------------
subject 14, num subject: 925, total acc: 96.8648681640625
pushup 115 95.65216827392578
squat 115 100.0
burpee 110 100.0
kneeup 105 100.0
yoga 150 100.0
cat 160 91.875
benchpress 170 93.5294189453125
----------------------------

In [29]:
subject_list, action_list

(array(['S11', 'S12', 'S11', ..., 'S18', 'S18', 'S18'], dtype='<U3'),
 array(['benchpress', 'pushup', 'kneeup', ..., 'cat', 'cat', 'cat'],
       dtype='<U10'))

In [27]:
dataset = readpkl('data/action/kookmin2_mmpose_train_s19-20_augment_243_50_test_augment_243_50.pkl')